# Filings Daily Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
import csv
from datetime import datetime

%load_ext sql
%config SqlMagic.displaylimit = 5

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
connect_to_db = 'postgresql://' + \
                os.getenv('PG_USER', '') + ":" + os.getenv('PG_PASSWORD', '') +'@' + \
                os.getenv('PG_HOST', '') + ':' + os.getenv('PG_PORT', '5432') + '/' + os.getenv('PG_DB_NAME', '');
    
%sql $connect_to_db

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql 
select now() AT TIME ZONE 'PST' as current_date

In [ ]:
%%sql coop_gazette_changeofname  <<
select
  to_char(f.effective_date,'MON dd, yyyy')
||';'
|| CASE WHEN b.legal_type='CP' THEN 'CP'||substr(b.identifier,6,4) ||'    '
        WHEN b.legal_type='XCP' THEN 'XCP'||substr(b.identifier,7,4)||'   '
        ELSE substr(b.identifier,0,10)||' ' 
   END  
||';'
||rpad(substr(old.legal_name,1,52),52)
||';'
||' to '
||';'
||rpad(substr(new.legal_name,1,58),58)
from filings      f
    ,businesses   b 
    ,businesses_version old 
    ,businesses_version new 
where f.filing_type in ('alteration', 'changeOfName')   
  and b.id=f.business_id
  and f.transaction_id=old.end_transaction_id 
  and f.transaction_id=new.transaction_id
  and f.source='LEAR' 
  and old.legal_name != new.legal_name
  and f.sent_to_gazette_date is null   
order by f.effective_date;

In [ ]:
datestr = datetime.strftime(datetime.now(), '%Y%m%d')
coop_gazette_changeofname_filename = os.getenv('DATA_DIR', '')+'COOP_GAZETTE_CHANGEOFNAME_' + datestr +'.TXT'

with open(coop_gazette_changeofname_filename, 'w') as f:
    if not coop_gazette_changeofname.DataFrame().empty:
        dfAsString = coop_gazette_changeofname.DataFrame().to_string(header=False, index=False)
        f.write(dfAsString)